In [113]:
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt

In [172]:
n_actions = 2
n_states = 12
episode_len = 20

In [173]:
def permutations(elems, T):
    def recurse(xs):
        if len(xs[0]) >= T:
            return xs
        else:
            return recurse([x+[e] for x in xs for e in elems]) # + xs
            
    return recurse([[e] for e in elems])

In [174]:
elems = list(range(n_actions))
trajectories = np.array(permutations(elems, episode_len))
n_trajectories = trajectories.shape[0]
print(n_trajectories)

1048576


In [175]:
trajectories  # think about these as sequences of actions or as sequences of states?

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [1, 1, 1, ..., 1, 0, 1],
       [1, 1, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 1, 1, 1]])

Assume rewards are given for being in state s.

(can I have an example of rewards as a fn of $r(s,a)$ state and action!? is it not just equivalent to $r(s_{t+1})$? not if transitions are stochastic. but surely we only reward the actionif it actually does something to the state!?!?

In [141]:
rewards = np.zeros((1, n_states))
rewards[0,1]= 1
rewards.shape

(1, 12)

In [155]:
# via policy evaluation
# a distribution over the possible trajectories

observed_rewards = rnd.choice([0.0,1.0], size=(1, n_trajectories), p=[0.9, 0.1])
# many trajectories might arrive in the same state and thus receive the same reward.

p_traj = np.exp(rnd.standard_normal((n_trajectories)))
p_traj /= p_traj.sum()
p_traj

# value = 

array([0.00035946, 0.00031307, 0.00314563, ..., 0.00359513, 0.00097901,
       0.00104118])

In [156]:
observed_rewards

array([[0., 0., 0., ..., 0., 1., 0.]])

Two trajectories can be factorised if;

$$
t_1 = t_2 \;\;\text{if}\\
\sum_{i=0}^t \gamma^t r(s_{t_1[i]}) = \sum_{i=0}^t \gamma^tr(s_{t_2[i]})  \\
\text{Invariance criteria} \\
V(t_1) = V(t_2) \\
s_{t_1[T]} = s_{t_2[T]} \\
$$

But, stochasticity changes everything... Unlikely even that $V(t_1) = V(t_1)$... or $s_{t_1[T]} = s_{t_1[T]}$

In [159]:
n_options = 8
options = np.arange(n_options)

# want to find a mapping from actions to options. a factorisation!?
# s.t. options = A.T = np.dot(abstraction, trajectories). 
# abstraction = (n_options x n_trajectories)
# s.t. the rows of A are sparse (only contain a 1 in one column)

In [163]:
trajectories.shape, options.shape

((1024, 5), (8,))

In [161]:
np.linalg.lstsq(trajectories, options)

LinAlgError: Incompatible dimensions

If the reward is independent of / invariant to a dimension is the state space then we can factorise the action space!!!

Eg. Eating candy. Still rewarded whether legs are folded or not. Or whether inside or out. Etc.

This we don't have to consider actions that only change the invariant dimensions.


Assume $r(\cdot)$ is invariant to noise / changes in the $i$th dimension. Aka, there exits $i$ s.t. $r(s) = r(s + ne_i), n\sim \mathcal N$.

This means that;
- any two (sub) trajectories that vary in only the $i$th dimension can be ???
- any action that only changes the state in the $i$th dimension can be temporally abstracted.

Ultimate goal is to find a single button/action/subgoal/option that maximises instantaneous and future reward. We can just keep hitting that button.

To see the future soo clearly that we can pick a policy for the next 10 years. And achieve max reward.

In this set of temporally abstracted options, there are may redundancies.

For example, imagine we have the action, step left (l), step right (r) and do nothing (n).
Then we might have the case where;

- llr =  rll = lrl = lnn = nln = nnl (ie the order doesnt matter).

But sometimes the order may matter. We might have stop (s), drop (d) and roll (r).



What other 'higher' order symmetries might there be within sequences of actions. Inter subset symmetries of actionscan be; 

- permuted (any permutation)
- mirrored (abcd = dcba)
- rotated (abcd = dabc)
- ?

And intra subset symmetries of actions can have symmetries

- mirrored (AasdsfgfB = BasdsfgfA) where A=sdf, B=rge

High freq setting.

Imagine a setting where you ahve access to hundreds of actions. But only a few actually move you through the maze (left, right, up, down). Each of these moves takes ~n steps to execute. Other (non movement) actions can be taken during their execution.

This problem is a lot harder (how much???) than the original (left, right up down).


A subgoal (that is to be reached in k steps) defines a subset of possible trajectories. 

In [1]:
def permutations(elems, T):
    def recurse(xs):
        if len(xs[0]) >= T:
            return xs
        else:
            return recurse([x+[e] for x in xs for e in elems]) # + xs
            
    return recurse([[e] for e in elems])

class RndOptionPolicy():
    def __init__(self, n_actions, n_time_steps):
        self.options = np.array(permutations(range(n_actions), n_time_steps))
        
    def __call__(self, x):
        rnd_idx = rnd.choice(np.arange(self.options.shape[0]))
        return self.options[rnd_idx]
     
class OptionEnvWrapper():
    def __init__(self, env):
        self.env = env
        
    def step(self, s, actions):
        R = 0
        for a in actions:
            s, r = self.env.step(s, a)
            R += r
        return s, R

    def reset(self):
        return self.env.reset()

In [2]:
# utils
def onehot(idx, N): # hacky. i know...
    return np.eye(N)[idx]

def softmax(x):
    return np.exp(x)/np.sum(np.exp(x))

class Env():
    def __init__(self, n_states, n_actions):
        self.n_states = n_states
        self.S = np.arange(n_states)
        self.A = np.arange(n_actions)

        # model the transitions as linear fns conditional on the action.
        # P = np.random.standard_normal([n_actions, n_states, n_states]) **2 # make sharper

        # deterministic transition fn
        # each action move from state(i) to state(j) with probability 1.
        # BUG nope. softmax doesnt do this. will need to set to -infty
        self.P = 20*np.stack([np.random.permutation(np.eye(n_states, dtype=np.float32)) for _ in range(n_actions-1)] + [np.eye(n_states, dtype=np.float32)],axis=0)  
        # TODO what if there is structure in P? Low rank? Shared over actions?
        # QUESTION how does this parameterisation effect things?
        # NOTE this graph might be disconnected. but is unlikely!?

        # reward is only a fn of the current state - shape = [n_states]
        # also. is sparse.
        self.R = onehot(np.random.randint(0, n_states), n_states)

    def step(self, state, action):
        """
        A tabular, probabilistic step function. 

        Args:
            state (int): An element of S. The current state
            state (int): An element of A. The action to be taken

        Returns:
            new_state (int): An element of S.
        """
        # step by selecting relevant transition matrix and applying
        logits = np.matmul(self.P[action, ...], onehot(state, self.n_states))
        # convert to a distribution and sample
        new_s = np.random.choice(self.S, p=softmax(logits))
        return new_s, self.R[new_s]
    
    def rnd_policy(self, s, *args):
        return np.random.choice(self.A)
    
    def reset(self):
        return np.random.choice(self.S)

    def new_task(self):
        self.R = onehot(np.random.randint(0, self.n_states), self.n_states)

In [3]:
n_actions = 4
n_states = 24
env = Env(n_states, n_actions)
rnd_policy = lambda obs: env.action_space.sample()
op = RndOptionPolicy(n_actions, 6)
env = OptionEnvWrapper(env)
len(op.options), op(2)

NameError: name 'np' is not defined

In [4]:
def play_episode(env, player, T=5):
    # reset
    s = env.reset()
    R = 0
    done = False
    pairs = []
    
    # play an episode
    for _ in range(10):

        a = player(s)
        new_s, r = env.step(s, a)
        R += r
        
        pairs.append((np.concatenate([onehot(new_s, n_states), np.array([0])]), 
                      a, 
                      np.concatenate([onehot(new_s, n_states), np.array([R])])))
        s = new_s
            
    return pairs

In [ ]:
def get_pairs():
    pairs = play_episode(env,op)
    pairs = tuple(zip(*pairs))
    return tuple([np.vstack(p) for p in pairs])

def get_n(n):
    pairs = [get_pairs() for i in range(n)]
    pairs = tuple(zip(*pairs))
    return tuple([np.vstack(p) for p in pairs])